In [6]:
#!/usr/bin/env python
#-- coding: utf-8 --
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import datetime
import pandas as pd
import numpy as np
import messages
from importlib import reload
import pandas as pd
import glob as gb
import numpy as np
from dateutil.relativedelta import relativedelta




In [7]:
class Air:

    def __init__(self,Variable=None,Estaciones=None,Fechai=None, Fechaf=None,Freq=5):

        self.Fechai      = (dt.datetime.now()-relativedelta(months=1)).strftime('%Y-%m-')+'01 01:00' if (Fechaf == None) else Fechai
        self.Fechaf      = (pd.to_datetime(self.Fechai)+ relativedelta(months=1)-dt.timedelta(hours=1)).strftime('%Y-%m-%d %H:%M') if (Fechaf == None) else Fechaf
        ##### Modificarlo si se cambia de pc  #####################################
        self.Est         = pd.read_csv('Estaciones.txt',index_col=0).sort_values(by='Latitud',ascending=False)
        self.Dropbox_path='/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/'
        ###########################################################################

        self.folder={'BAR':'01_BARBOSA/','BEL':'02_BELLO/','CAL':'03_CALDAS/','COP':'04_COPACABANA/','GIR':'05_GIRARDOTA/','ITA':'06_ITAGUI/','SUR':'07_LA ESTRELLA/','EST':'07_LA ESTRELLA/','MED':'08_MEDELLIN/','CEN':'08_MEDELLIN/','SAB':'09_SABANETA/','ENV':'10_ENVIGADO/'}
        self.nombre_tipo={'MULTIPUNTO':'MP','VER. CERO':'VC','CERO_SPAN PREC.':'CS_PR'}
        self.fecha_range =pd.date_range(self.Fechai,self.Fechaf,freq='M')
    def Read_Verification(self,*args,**kwargs):
        self.var		= kwargs.get('var','NOx')
        self.tipo		= kwargs.get('tipo','CS') #CS: ZERO/SPAN, 'PRE'

        self.C={'MP':{},'VC':{},'CS_PR':{}}

        analizador= 'O3(A)' if self.var=='Ozono' else 'NOX(A)' if self.var=='NOx' else 'CO(A)' if self.var=='CO' else 'SO2(A)' if self.var=='SO2' else None
        self.estaciones=self.Est['Nombre'][self.Est[self.var]==1].values

        for est in self.estaciones:
            for date in self.fecha_range:
                #print (date)
                #print (est)
                #print (self.Dropbox_path+self.folder[est[:3]]+est+'/'+analizador+'/VERIFICACIONES/'+date.strftime('%Y/%m')+'*')
                self.foldermes=gb.glob(self.Dropbox_path+self.folder[est[:3]]+est+'/'+analizador+'/VERIFICACIONES/'+date.strftime('%Y/%m')+'*')
                if len(self.foldermes)>0:
                    XLS=gb.glob(self.foldermes[0]+'/F*.xlsx')
                    print (XLS)

                    for name in XLS:
                        print (name)
                        self.xl = pd.ExcelFile(name)
                        self.DF = self.xl.parse(0)
                        print('++++++++++ calcula cada variable por archivo ++++++')
        for tipofile in self.C.keys():
            print(tipofile)
            print ('****************tipofile*******************')

            
            
            

In [8]:
Fechai = '2022-07-01'
Fechaf='2022-08-01'
self=Air(Fechai=Fechai,Fechaf=Fechaf)

In [9]:
DFTipoFechaEst = pd.DataFrame()
Fecha          = []
VAR=(['NOx','CO', 'SO2', 'Ozono'])

for k, var in enumerate(VAR):

    self.Read_Verification(var=var)
    print (var)
    analizador= 'O3(A)' if self.var=='Ozono' else 'NOX(A)' if self.var=='NOx' else 'CO(A)' if self.var=='CO' else 'SO2(A)' if self.var=='SO2' else None

    for tipofile in self.C.keys():
        print(tipofile)
        print ('****************tipofile*******************')

        for est in self.estaciones:
            for date in self.fecha_range:
                    #print (date)
                    #print (est)
                    #print (self.Dropbox_path+self.folder[est[:3]]+est+'/'+analizador+'/VERIFICACIONES/'+date.strftime('%Y/%m')+'*')
                    self.foldermes=gb.glob(self.Dropbox_path+self.folder[est[:3]]+est+'/'+analizador+'/VERIFICACIONES/'+date.strftime('%Y/%m')+'*')
                    if len(self.foldermes)>0:
                        XLS=gb.glob(self.foldermes[0]+'/F*.xlsx')
                        print (XLS)

                        for name in XLS:
                            print (name)
                            self.xl = pd.ExcelFile(name)
                            self.DF = self.xl.parse(0)

                            dondeResultados=np.where(self.DF['Unnamed: 1'].str.find('7. ')==0)[0]
                            Resultados = self.xl.parse(0, skiprows=dondeResultados[0]+1,nrows=7,header=2,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')
                            dondeTipo=np.where(self.DF['Unnamed: 1'].str.find('4. ')==0)[0]
                            dondeFecha = np.where(self.DF['Unnamed: 1'].str.find('1. ')==0)[0]

                            Estacion = self.xl.parse(0, skiprows=dondeFecha[0]+1,nrows=2,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][0]

                            try:
                                Tipo  =  self.xl.parse(0, skiprows=dondeTipo[0]+1,nrows=1,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][0]
                                try:
                                    Fecha =  self.xl.parse(0, skiprows=dondeFecha[0]+3,nrows=2,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 6'][0]
                                    Hora_inicio = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][2]
                                    Hora_fin = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 5'][2]

                                except:

                                    Fecha = self.xl.parse(0, skiprows=dondeFecha[0]+3,nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 9'][0]
                                    Hora_inicio = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][2]
                                    Hora_fin = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 5'][2]


                            except: 
                                print (u'Entró al except en la variable:')
                                print (var)
                                print (u'En la estación:')
                                print (est)
                                #dondeTipo=np.where(self.DF['Unnamed: 1'].str.find('4. ')==0)[0]
                                Tipo  =  self.xl.parse(0, skiprows=dondeTipo[0]+1,nrows=1,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 3'][0]
                                #dondeFecha = np.where(self.DF['Unnamed: 1'].str.find('1. ')==0)[0]
                                try:
                                    Fecha =  self.xl.parse(0, skiprows=dondeFecha[0]+3,nrows=2,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 6'][0]
                                    Hora_inicio = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][2]
                                    Hora_fin = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 5'][2]

                                except:

                                    Fecha = self.xl.parse(0, skiprows=dondeFecha[0]+3,nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 9'][0]
                                    Hora_inicio = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][2]
                                    Hora_fin = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 5'][2]
     
                                #Fecha      = Fecha.append(FechaDummy)
                            
                            print (Tipo)
                            print ('Tipo:   *******************')
                            TipoVerif = 'CS_PR' if (Tipo=='CERO_SPAN PREC.' and tipofile == 'CS_PR' ) else 'MP' if (Tipo=='MULTIPUNTO' and tipofile == 'MP' )  else None
                            if (Tipo=='CERO_SPAN PREC.' and tipofile == 'CS_PR' ) or (Tipo=='MULTIPUNTO' and tipofile == 'MP' ):
                                DFDummy   = pd.DataFrame()

                                DFDummy   = pd.DataFrame([{
                                'Variable':var,
                                'Tipo': TipoVerif,
                                'Fecha': Fecha,
                                'Hora_inicio': Hora_inicio,
                                'Hora_fin' : Hora_fin,
                                'Estacion':est}])
                                DFTipoFechaEst = DFTipoFechaEst.append(DFDummy)
                                
        print (DFTipoFechaEst)

DFTipoFechaEst.to_csv('Verificaciones_Contaminantes_Estacion{0}_{1}.csv'.format(Fechai,Fechaf))    


['/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/05_GIRARDOTA/GIR-EPM/NOX(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-21_GIR-EPM_NOx_CSPR_2022-07-21.xlsx', '/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/05_GIRARDOTA/GIR-EPM/NOX(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-22_GIR-EPM_NOx_MP_2022-07-07.xlsx']
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/05_GIRARDOTA/GIR-EPM/NOX(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-21_GIR-EPM_NOx_CSPR_2022-07-21.xlsx
++++++++++ calcula cada variable por archivo ++++++
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/05_GIRARDOTA/GIR-EPM/NOX(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-22_GIR-EPM_NOx_MP_2022-07-07.xlsx


/Users/siata/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


++++++++++ calcula cada variable por archivo ++++++
['/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-ITMR/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-ITMR_NOx_CSPR_2022-07-12.xlsx', '/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-ITMR/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-ITMR_NOx_CSPR_2022-07-26.xlsx']
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-ITMR/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-ITMR_NOx_CSPR_2022-07-12.xlsx
++++++++++ calcula cada variable por archivo ++++++
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-ITMR/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-ITMR_NOx_CSPR_2022-07-2

/Users/siata/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


++++++++++ calcula cada variable por archivo ++++++
['/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-FISC/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-FISC_NOx_SPCR_2022-07-25.xlsx', '/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-FISC/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-FISC_NOx_SPCR_2022-07-12.xlsx']
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-FISC/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-FISC_NOx_SPCR_2022-07-25.xlsx
++++++++++ calcula cada variable por archivo ++++++
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-FISC/NOX(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-22_MED-FISC_NOx_SPCR_2022-07-1

...

Entró al except en la variable:
Ozono
En la estación:
MED-MIRA
CERO_SPAN PREC.
Tipo:   *******************
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-MIRA/O3(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-32_MED-MIRA_O3_CSPR_2022-07-13.xlsx
Entró al except en la variable:
Ozono
En la estación:
MED-MIRA
CERO_SPAN PREC.
Tipo:   *******************
['/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-LAYE/O3(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-32_MED-LAYE_O3_CSPR_2022-07-28.xlsx', '/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-LAYE/O3(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-32_MED-LAYE_O3_CSPR_2022-07-14.xlsx']
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES 

/Users/siata/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Entró al except en la variable:
Ozono
En la estación:
MED-LAYE
CERO_SPAN PREC.
Tipo:   *******************
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/08_MEDELLIN/MED-LAYE/O3(A)/VERIFICACIONES/2022/07_JULIO/F-GAA-RA-32_MED-LAYE_O3_CSPR_2022-07-14.xlsx
Entró al except en la variable:
Ozono
En la estación:
MED-LAYE
CERO_SPAN PREC.
Tipo:   *******************
['/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/06_ITAGUI/ITA-CONC/O3(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-32_ITA-CONC_O3_CSPR_2022-07-12.xlsx', '/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/06_ITAGUI/ITA-CONC/O3(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-32_ITA-CONC_O3_CSPR_2022-07-26.xlsx']
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE A

/Users/siata/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


Entró al except en la variable:
Ozono
En la estación:
ITA-CONC
CERO_SPAN PREC.
Tipo:   *******************
/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/06_ITAGUI/ITA-CONC/O3(A)/VERIFICACIONES/2022/07_Julio/F-GAA-RA-32_ITA-CONC_O3_CSPR_2022-07-26.xlsx
Entró al except en la variable:
Ozono
En la estación:
ITA-CONC
CERO_SPAN PREC.
Tipo:   *******************
  Variable   Tipo      Fecha Hora_inicio  Hora_fin  Estacion
0      NOx     MP 2022-07-07    11:40:00  14:30:00   GIR-EPM
0      NOx     MP 2022-07-05    11:50:00  15:30:00  CEN-TRAF
0      NOx     MP 2022-07-07    10:00:00  15:00:00  SUR-TRAF
0      NOx  CS_PR 2022-07-21    09:40:00  11:20:00   GIR-EPM
0      NOx  CS_PR 2022-07-12    13:05:00  16:24:00  MED-ITMR
0      NOx  CS_PR 2022-07-26    09:09:00  13:45:00  MED-ITMR
0      NOx  CS_PR 2022-07-25    09:03:00  12:34:00  MED-FISC
0      NOx  CS_PR 2022-07-12    09:45:00  11:35:00  MED-FISC
0      NOx  CS_PR

In [10]:
gb.glob(self.Dropbox_path+self.folder[est[:3]]+est+'/'+analizador+'/VERIFICACIONES/'+date.strftime('%Y/%m')+'*')
                    

['/Users/siata/SIATA Dropbox/SGC_Siata_ Aire/01_Gestion_Operativa/03_Registros/HojasVida_FichasTecnicas/ESTACIONES DE AIRE/06_ITAGUI/ITA-CONC/O3(A)/VERIFICACIONES/2022/07_Julio']

In [ ]:
self.xl.parse(0, skiprows=dondeFecha[0]+1,nrows=2,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')


In [ ]:
self.DF

In [ ]:
dondeResultados=np.where(self.DF['Unnamed: 1'].str.find('7. ')==0)[0]

In [ ]:
dondeResultados

In [ ]:
Resultados = self.xl.parse(0, skiprows=dondei[0]+1,nrows=7,header=2,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')
            

In [ ]:
dondeTipo=np.where(self.DF['Unnamed: 1'].str.find('4. ')==0)[0]
Tipo =  self.xl.parse(0, skiprows=dondeTipo[0]+1,nrows=1,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 3'][0]
dondeFecha = np.where(self.DF['Unnamed: 1'].str.find('1. ')==0)[0]
Fecha = self.xl.parse(0, skiprows=dondeFecha[0]+3,nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 9'][0]


In [ ]:
dondeTipo=np.where(self.DF['Unnamed: 1'].str.find('4. ')==0)[0]

In [ ]:
np.where(self.DF['Unnamed: 1'].str.find('1. ')==0)[0]


In [ ]:
Tipo =  self.xl.parse(0, skiprows=dondeTipo[0]+1,nrows=1,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][0]
       

In [ ]:
Tipo

In [ ]:
self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')


In [ ]:
Fecha = self.xl.parse(0, skiprows=dondeFecha[0]+3,nrows=2,header=0,usecols=np.arange(10)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 6'][0]


In [ ]:
Hora_Inicio = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 2'][2]
Hora_Fin = self.xl.parse(0, skiprows=dondeFecha[0],nrows=4,header=0,usecols=np.arange(11)).dropna(axis=1, how='all').dropna(axis=0, how='all')['Unnamed: 5'][2]


In [ ]:
DFPrueba = pd.DataFrame([{
    'Variable':VAR[0],
    'Tipo': Tipo,
    'Fecha': Fecha,
    'Estacion':Estacion}])

In [ ]:
DFPrueba 

In [ ]:
DFPrueba = pd.DataFrame([{
    'Variable':VAR[1],
    'Tipo': Tipo,
    'Fecha': Fecha,
    'Estacion':Estacion}])

In [ ]:
DFPrueba = DFPrueba.append( pd.DataFrame([{
    'Variable':VAR[0],
    'Tipo': Tipo,
    'Fecha': Fecha,
    'Estacion':Estacion}]))

In [ ]:
self.DFTipoFechaEst